In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### 패키지 설치 & 라이브러리 import

In [ ]:
!pip install optuna
!pip install torchsummary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

## 전체 공통 함수

### 1. 데이터 보간 함수

- 데이터의 시간 간격을 0.5초의 등간격으로 맞추고 그에 따른 좌표 값을 보간함
- 가장 많이 사용되는 3차 스플라인 보간 적용

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt

# 데이터 보간 - 위도, 고도, 경도 값을 0.5초 시간 간격으로 보간
def spline_interpolation(x, y):
    # x: 보간할 데이터의 x 값 (시간)
    # y: 보간할 데이터의 y 값 (위도, 고도, 경도)

    # 3차 스플라인 보간 함수 생성
    f = interp1d(x, y, kind='cubic')

    # 시간 범위 설정
    start_time = min(x)
    end_time = max(x)

    # 0.5초 간격의 시간 배열을 생성
    new_x = np.arange(start_time, end_time, 0.5)

    # 위도, 경도, 고도 보간
    new_y = f(new_x)

    return new_x, new_y

### 2. 시퀀스 나누는 함수

- 학습을 위해 데이터를 시퀀스 형태로 나눔
- look back과 forward에 따라 학습에 사용될 x값의 구간과 y값을 반환함
- 초기 look back은 10, forward는 0이다.

In [ ]:
# 시퀀스 생성 함수
# look back : 10 (5초간의 위도, 고도, 경도 데이터를 x값으로 사용)
# forward : 0 (0.5초 뒤의 위도, 고도, 경도 데이터를 y값으로 사용)
def create_sequences(df, look_back=10, forward=0):
    xs = []
    ys = []
    for i in range(len(df) - look_back - forward):
        x = df.iloc[i:(i+look_back)].values
        y = df.iloc[i+look_back+forward].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

### 3. 시간 소수점 절사 함수

- 불필요한 시간 소수점 제거

In [ ]:
import math

# 시간 소수점 절사 함수
def truncation(df):
    new_time = []

    for a in df['time']:
      new_time.append(math.floor(a * 10) / 10)

    df['time'] = new_time

    return df

### 4. 스케일링 함수

- MinMaxScaling 적용 함수


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaling 적용 함수
def min_max_scaling(df):

    # time 삭제 df 생성
    mid_df=df.drop(columns='time')

    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(mid_df)

    min_max_data = min_max_scaler.transform(mid_df)

    new_data = pd.DataFrame(min_max_data)

    # time 추가
    new_data.insert(0, 'time', df['time'])

    # 나머지 열의 이름 가져오기
    column_names = mid_df.columns.tolist()

    column_names.insert(0, 'time')

    print(column_names)

    # 새로운 데이터프레임에 열 이름 설정
    new_data.columns = column_names

    return new_data

### 5. train/validation/test 분리 함수

- Train, Validation, Test 데이터셋을 분리하는 함수이다.
- 각각의 비율은 6:2:2 이다.

In [ ]:
from sklearn.model_selection import train_test_split

# train/validation/test 분리 함수
def split_train_val_test(X, y):

    #Train, Test 분류
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #Train, Validation 분류
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)


    # 데이터 크기 출력
    print("Train set:", X_train.shape, y_train.shape)
    print("Validation set:", X_val.shape, y_val.shape)
    print("Test set:", X_test.shape, y_test.shape)

    return X_train, X_val, X_test, y_train, y_val, y_test

### 6. 경로 시각화 함수 (상의 필요)

- 실제 경로와 예측 경로를 각각 그래프로 생성

In [ ]:
import matplotlib.pyplot as plt

# 경로 시각화 함수
def prediction_visualization(y_test, y_pred):
    # 시각화를 위해 위도, 경도, 고도 분리
    y_test_lat = y_test[:, 0]
    y_test_lon = y_test[:, 1]
    y_test_alt = y_test[:, 2]

    y_pred_lat = y_pred[:, 0]
    y_pred_lon = y_pred[:, 1]
    y_pred_alt = y_pred[:, 2]

    # 3D 그래프 생성
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # 실제 값 플롯
    ax.scatter(y_test_lat, y_test_lon, y_test_alt, c='b', marker='o', label='Actual')

    # 예측 값 플롯
    ax.scatter(y_pred_lat, y_pred_lon, y_pred_alt, c='r', marker='^', label='Predicted')

    # 그래프 제목 및 축 레이블 설정
    ax.set_title('Actual vs Predicted')
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Longitude')
    ax.set_zlabel('Altitude')

    # 범례
    ax.legend()

    # 그래프를 화면에 출력
    plt.show()

## 전처리

### 1. 데이터셋 가져오기

In [ ]:
import pandas as pd

# 데이터셋 가져오기 (필요시 저장 경로 수정)
raw = pd.read_csv("/content/OnboardGPS.csv")

In [ ]:
# 데이터셋 출력
raw

,Timpstemp,imgid,lat,lon,alt,s_variance_m_s,c_variance_rad,fix_type,eph_m,epv_m,vel_n_m_s,vel_e_m_s,vel_d_m_s,num_sat,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,7009129,1,47.384357,8.545178,464.910,1.06,2.163928,3,12.992001,1.750000e-43,0.24,0.17,0.21,6,NaN,NaN,NaN,
1,7042462,2,47.384357,8.545178,464.910,1.06,2.163928,3,12.992001,1.750000e-43,0.24,0.17,0.21,6,NaN,NaN,NaN,
2,7075795,3,47.384357,8.545178,464.910,1.06,2.163928,3,12.992001,1.750000e-43,0.24,0.17,0.21,6,NaN,NaN,NaN,
3,7109128,4,47.384357,8.545178,464.910,1.06,2.163928,3,12.992001,1.750000e-43,0.24,0.17,0.21,6,NaN,NaN,NaN,
4,7142461,5,47.384357,8.545178,464.910,1.06,2.163928,3,12.992001,1.750000e-43,0.24,0.17,0.21,6,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81164,2720555185,81165,47.383788,8.546155,480.784,1.25,0.766510,3,9.997001,1.300000e-43,1.02,-0.17,0.07,8,NaN,NaN,NaN,
81165,2720588518,81166,47.383788,8.546155,480.784,1.25,0.766510,3,9.997001,1.300000e-43,1.02,-0.17,0.07,8,NaN,NaN,NaN,
81166,2720625687,81167,47.383789,8.546155,480.567,1.40,0.766560,3,10.004001,1.300000e-43,0.88,0.14,-0.04,8,NaN,NaN,NaN,
81167,2720659020,81168,47.383789,8.546155,480.567,1.40,0.766560,3,10.004001,1.300000e-43,0.88,0.14,-0.04,8,NaN,NaN,NaN,


### 2. 데이터셋 전처리

- 데이터셋 보간

In [ ]:
tri = raw[['Timpstemp', ' lat', ' lon', ' alt']]

new_x, lat = spline_interpolation(tri['Timpstemp'] / 1e6 , tri[' lat'])
new_x, lon = spline_interpolation(tri['Timpstemp'] / 1e6 , tri[' lon'])
new_x, alt = spline_interpolation(tri['Timpstemp'] / 1e6 , tri[' alt'])

tri = pd.DataFrame({"time":new_x, "lat":lat, "lon":lon, "alt":alt})

tri

,time,lat,lon,alt
0,7.009129,47.384357,8.545178,464.910000
1,7.509129,47.384353,8.545179,465.481943
2,8.009129,47.384350,8.545184,466.333961
3,8.509129,47.384350,8.545184,466.897962
4,9.009129,47.384350,8.545190,467.267636
...,...,...,...,...
5423,2718.509129,47.383776,8.546159,481.215995
5424,2719.009129,47.383779,8.546158,480.987856
5425,2719.509129,47.383782,8.546157,480.716495
5426,2720.009129,47.383784,8.546156,480.986321


- 소수점 절사


In [ ]:
tri = truncation(tri)
tri

,time,lat,lon,alt
0,7.0,47.384357,8.545178,464.910000
1,7.5,47.384353,8.545179,465.481943
2,8.0,47.384350,8.545184,466.333961
3,8.5,47.384350,8.545184,466.897962
4,9.0,47.384350,8.545190,467.267636
...,...,...,...,...
5423,2718.5,47.383776,8.546159,481.215995
5424,2719.0,47.383779,8.546158,480.987856
5425,2719.5,47.383782,8.546157,480.716495
5426,2720.0,47.383784,8.546156,480.986321


- MinMaxScaling 적용

In [ ]:
tri = min_max_scaling(tri)
tri

['time', 'lat', 'lon', 'alt']


,time,lat,lon,alt
0,7.0,0.414443,0.542298,0.222958
1,7.5,0.413652,0.542468,0.231141
2,8.0,0.413026,0.543388,0.243331
3,8.5,0.413007,0.543482,0.251401
4,9.0,0.413008,0.544756,0.256690
...,...,...,...,...
5423,2718.5,0.300069,0.748155,0.456257
5424,2719.0,0.300766,0.747924,0.452992
5425,2719.5,0.301346,0.747820,0.449110
5426,2720.0,0.301690,0.747511,0.452970


## 모델 선언

### 1. 모델 클래스 선언
- LSTM을 사용한 모델 클래스 선언
- 구조 : LSTM + Fully Connected Layer

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import optuna

# LSTM 모델 클래스 정의
class LSTM_Model(nn.Module):
    def __init__(self, input_size=3, hidden_size=128, num_layers=2, output_size=3):
        super(LSTM_Model, self).__init__()

        # hidden state size
        self.hidden_size = hidden_size

        # LSTM layer 수
        self.num_layers = num_layers

        # LSTM 레이어
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # LSTM hidden state, cell state 초기화
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM 레이어를 통과
        out, _ = self.lstm(x, (h0, c0))

        # 마지막 시점의 hidden state에서 Fully Connected Layer를 통과
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
# 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 공간 정의
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 16, 64)


    num_epochs = 10

    input_size = X_train.shape[1]  # 입력 특징 수 -> 3 (lat, lon, alt)
    output_size = y_train.shape[1]  # 출력 특징 수 -> 3 (lat, lon, alt)

    # 데이터셋 로더
    train_loader, val_loader, _ = create_dataloaders(X_train, y_train, batch_size)

    # 모델 정의: LSTMModel
    model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()

    # epoch 따로 빼기
    # 모델 학습

    for epoch in range(num_epochs):  # 10 에포크 동안 학습
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            # 입력 데이터의 차원을 맞추기 위해 (batch_size, 1, input_size) 형태로 변환
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

    # Validation set에서 성능 측정
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.to(device)  # 입력 데이터 차원 조정
            batch_y = batch_y.to(device)
            output = model(batch_x)
            loss = criterion(output, batch_y)
            val_loss += loss.item()

    # Optuna에 검증 손실 리포트
    trial.report(val_loss, epoch)

    # Early stopping 조건
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return val_loss
    # return val_loss / len(val_loader)

## 학습

### 1. 데이터 준비

1. 시퀀스 생성
2. Train, Validation, Test 분리
3. DataLoader에 load

In [ ]:
### 같이 수정 필요

# 데이터 준비
def create_dataloaders(tri, batch_size):

  tri = tri[['time', 'lat', 'lon', 'alt']]

  train_x, train_y = create_sequences(tri[['lat', 'lon', 'alt']], sequence_length)

  X_train, X_val, X_test, y_train, y_val, y_test = split_train_val_test(train_x, train_y)

  X_train = torch.tensor(X_train, dtype=torch.float32)
  y_train = torch.tensor(y_train, dtype=torch.float32)
  X_val = torch.tensor(X_val, dtype=torch.float32)
  y_val = torch.tensor(y_val, dtype=torch.float32)
  X_test = torch.tensor(X_test, dtype=torch.float32)
  y_test = torch.tensor(y_test, dtype=torch.float32)

  train_dataset = TensorDataset(X_train, y_train)
  validation_dataset = TensorDataset(X_val, y_val)
  test_dataset = TensorDataset(X_test, y_test)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader, test_loader

Train set: (3255, 3) (3255, 3)
Validation set: (1086, 3) (1086, 3)
Test set: (1086, 3) (1086, 3)


### 2. 최적의 하이퍼파라미터 탐색

#### 1. 하이퍼 파라미터 탐색

In [ ]:
import optuna

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# 최적의 하이퍼파라미터 출력
print("Best hyperparameters: ", study.best_params)

[I 2024-09-22 14:02:00,132] A new study created in memory with name: no-name-1446028a-82d3-4fe2-a4b1-aec1901e38b1
[I 2024-09-22 14:02:14,732] Trial 0 finished with value: 0.0018377087424810538 and parameters: {'hidden_size': 92, 'num_layers': 4, 'dropout_rate': 0.40758179865115407}. Best is trial 0 with value: 0.0018377087424810538.
[I 2024-09-22 14:02:18,047] Trial 1 finished with value: 0.0005782707397456226 and parameters: {'hidden_size': 52, 'num_layers': 1, 'dropout_rate': 0.26059854028496715}. Best is trial 1 with value: 0.0005782707397456226.
[I 2024-09-22 14:03:02,927] Trial 2 finished with value: 0.0015939974520519813 and parameters: {'hidden_size': 241, 'num_layers': 4, 'dropout_rate': 0.4322841018199214}. Best is trial 1 with value: 0.0005782707397456226.
[I 2024-09-22 14:03:10,364] Trial 3 finished with value: 0.002140212891390547 and parameters: {'hidden_size': 32, 'num_layers': 4, 'dropout_rate': 0.44601513670050974}. Best is trial 1 with value: 0.0005782707397456226.
[I 

Best hyperparameters:  {'hidden_size': 74, 'num_layers': 1, 'dropout_rate': 0.21888626721684276}


#### 2. 하이퍼 파라미터 저장

In [ ]:
### 하이퍼 파라미터 저장

# 최적의 하이퍼파라미터를 JSON 파일로 저장
with open("best_hyperparameters.json", "w") as f:
    json.dump(best_params, f, indent=4)

print("Best hyperparameters have been saved to 'best_hyperparameters.json'")

### 3. 최적의 파라미터로 모델 학습

In [ ]:
# 최적의 모델 학습
best_params = study.best_params

train_loader, val_loader, test_loader = create_dataloaders(tri, best_params["batch_size"])

best_model = LSTMModel(input_size=X_train.shape[1], hidden_size=best_params['hidden_size'],
                      num_layers=best_params['num_layers'], output_size=y_train.shape[1],)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model = best_model.to(device)

optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params["learning_rate"])
criterion = torch.nn.MSELoss()

In [ ]:
print("Model Architecture and Parameter Count:")
summary(best_model, input_size=(3, 50))  # 입력 사이즈 (채널 수, 시퀀스 길이)로 수정

In [ ]:
import time
import torch

best_model.train()
total_train_time = 0  # 총 학습 시간 기록 변수

num_epochs = 50
for epoch in range(num_epochs):  # 최종 모델 학습
    model.train()
    train_loss = 0.0
    epoch_start_time = time.time()  # 에포크 시작 시간 기록
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)  # 입력 데이터 차원 조정
        batch_y = batch_y.to(device)
        optimizer.zero_grad()
        output = best_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    epoch_end_time = time.time()  # 에포크 종료 시간 기록
    epoch_duration = epoch_end_time - epoch_start_time  # 한 에포크의 학습 시간
    total_train_time += epoch_duration  # 총 학습 시간에 추가

    # Validation set에서 성능 측정
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.to(device)  # 입력 데이터 차원 조정
            batch_y = batch_y.to(device)
            output = model(batch_x)
            loss = criterion(output, batch_y)
            val_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # 모델 저장
    model_save_path = f"best_model_epoch_{epoch+1}.pth"
    torch.save(best_model.state_dict(), model_save_path)
    print(f"Model saved: {model_save_path}")

    print(f"Epoch {epoch+1}/{50} completed in {epoch_duration:.2f} seconds.")

print(f"Total training time: {total_train_time:.2f} seconds.")


## 테스트

### 1. 테스트 결과 생성

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch

# 모델 예측 및 성능 평가
best_model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    # 입력 데이터 차원 조정 (batch_size, seq_length, input_dim)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    # 예측
    y_pred_tensor = best_model(X_test_tensor)
    y_pred = y_pred_tensor.cpu().numpy()  # 예측 값을 numpy 배열로 변환

# 실제 값도 numpy 배열로 변환
y_true = y_test.numpy()

# 성능 평가
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')


Mean Squared Error (MSE): 2.5797276975936256e-05
Mean Absolute Error (MAE): 0.002969068707898259


<ipython-input-50-8fa169c05dea>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1).to(device)


### 2. 결과 시각화

In [ ]:
prediction_visualization(y_val, y_pred)

Todo
- min max scaling 풀어서, raw랑 찍어보기
- epoch이랑 batch, learning rate 찍어보기
- 파라미터 수 측정
- 모델 구조 출려 코드